In [ ]:
real_score = []
fake_score = []
epochs = 88
losses_g = []
losses_d = []
for epoch in range(epochs):
  loss_d_tmp = []
  loss_g_tmp = []
  real_score_tmp = []
  fake_score_tmp = []
  for real_image, i in tqdm(train_loader):
    netDis.zero_grad()
    real_image = real_image.to(device)
    real_image = Variable(real_image)
    real_preds = netDis(real_image)
    real_targets = Variable(torch.ones(real_image.size()[0], device=device)) 
    real_loss = criterion(real_preds, real_targets)
    latent = Variable(torch.randn(real_image.size()[0], latent_size, 1, 1, device=device))
    fake_image = netGen(latent)
    fake_targets = Variable(torch.zeros(real_image.size()[0], device=device))
    fake_preds = netDis(fake_image)
    fake_loss = criterion(fake_preds, fake_targets)
    real_score_tmp.append(torch.mean(real_preds).item())
    fake_score_tmp.append(torch.mean(fake_preds).item())
    loss_d = real_loss + fake_loss
    loss_d.backward()
    optDis.step()
    loss_d_tmp.append(loss_d.item())
    netGen.zero_grad()
    latent = torch.randn(real_image.size()[0], latent_size, 1, 1, device=device)
    fake_image = netGen(latent)
    preds = netDis(fake_image)
    targets = Variable(torch.ones(real_image.size()[0], device=device))
    loss_g = criterion(preds, targets)
    loss_g.backward()
    optGen.step()
    loss_g_tmp.append(loss_g.item()) 
    fake_image = netGen(latent) 
    vutils.save_image(fake_image.data, '%s/fake_samples_epoch_%03d.png' % ("/content/data", epoch), normalize = True)
    torch.save(netGen, "model_gen" + str(epoch) + ".zip")
    torch.save(netDis, "model_dis" + str(epoch) + ".zip")
  print(f'loss generator on {epoch} = {np.mean(loss_g_tmp)}')
  print(f'real scores on {epoch} = {np.mean(real_score_tmp)}')
  print(f'fake scores on {epoch} = {np.mean(fake_score_tmp)}')
  losses_g.append(np.mean(loss_g_tmp))
  losses_d.append(np.mean(loss_d_tmp))
  real_score.append(np.mean(real_score_tmp))
  fake_score.append(np.mean(fake_score_tmp))